In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import math
import nibabel as nib
from nilearn import image
from scipy import ndimage
import glob
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
from tensorflow.keras.preprocessing.image import load_img
import shutil
import itertools

In [7]:
from sklearn.metrics import classification_report

In [8]:
def resize_data(data):
    initial_size_x = data.shape[0]
    initial_size_y = data.shape[1]
    initial_size_z = data.shape[2]

    new_size_x = 128
    new_size_y = 128
    new_size_z = 128

    delta_x = initial_size_x / new_size_x
    delta_y = initial_size_y / new_size_y
    delta_z = initial_size_z / new_size_z

    new_data = np.zeros((new_size_x, new_size_y, new_size_z))

    for x, y, z in itertools.product(range(new_size_x),
                                     range(new_size_y),
                                     range(new_size_z)):
        new_data[x][y][z] = data[int(x * delta_x)][int(y * delta_y)][int(z * delta_z)]

    return new_data

In [9]:
def preprocess_anatomical(img_data_array):
    brain_volume = []
    highest_dim = max(img_data_array.shape)
    modify_channel_start = int(highest_dim/4)
    modify_channel_end = int(min(img_data_array.shape))
    stop_point = modify_channel_start + modify_channel_end

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume.append(temp)

    for i in range(img_data_array.shape[2]):
        img_slice_array_ana = img_data_array[:,:,i]
    #     img_slice_array_ana  = ndimage.rotate(img_slice_array_ana, 90)
        brain_volume.append(img_slice_array_ana)

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume.append(temp)
        
    brain_volume = np.array(brain_volume)
    brain_volume = brain_volume[40:214, 40:214, 40:214] 
    brain_volume = resize_data(brain_volume)
    return brain_volume

In [10]:
def preprocess_ground_truth(img_data_array_gt):
    brain_volume_gt = []
    highest_dim = max(img_data_array_gt.shape)
    modify_channel_start = int(highest_dim/4)
    modify_channel_end = int(min(img_data_array_gt.shape))
    stop_point = modify_channel_start + modify_channel_end

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume_gt.append(temp)

    for i in range(img_data_array_gt.shape[1]):
        img_slice_array_ana = img_data_array_gt[:,i,:]
    #     img_slice_array_ana  = ndimage.rotate(img_slice_array_ana, 90)
        brain_volume_gt.append(img_slice_array_ana)

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume_gt.append(temp)
        
    brain_volume_gt = np.array(brain_volume_gt)
    brain_volume_gt = brain_volume_gt[40:214,40:214, 40:214]
    brain_volume_gt = resize_data(brain_volume_gt)
    
    return  brain_volume_gt

In [11]:
def list_image_paths(directory_paths):
    image_paths = []
    for directory in range(len(directory_paths)):
        image_filenames = os.listdir(directory_paths[directory])
        for image_filename in image_filenames:
            image_paths.append(directory_paths[directory] + image_filename)
    return image_paths

In [12]:
# For anatomical slices
file_path = 'add a path'
files  =  os.listdir(file_path)
files  = [file for file in files if  file.endswith('.gz')]
files = sorted(files)
paths_ana  = [file_path + '/' + file for file in files]

# For ground truth
file_path_gt = 'add a path'
files  =  os.listdir(file_path_gt)
files  = [file for file in files if  file.endswith('.gz')]
files = sorted(files)
paths_gt  = [file_path_gt + '/' + file for file in files]

In [13]:
#     print("Now preparing image and masks number: ", index)
img_anatomical = nib.load(paths_ana[0])
retreve_volume = image.index_img(img_anatomical, 0)
# Get data from nibabel image object 
img_data = retreve_volume.get_data()
img_data_array = np.asarray(img_data)

In [15]:
img_data.shape, img_data_array.shape

((256, 256, 128), (256, 256, 128))

In [13]:
mri_volumes, ground_truth_volumes = [], []
for i in range(len(paths_ana)):
    # anatomical slice preparing
    index = i+1
#     print("Now preparing image and masks number: ", index)
    img_anatomical = nib.load(paths_ana[i])
    retreve_volume = image.index_img(img_anatomical, 0)
    # Get data from nibabel image object 
    img_data = retreve_volume.get_data()
    img_data_array = np.asarray(img_data)
    pre_anatomical = preprocess_anatomical(img_data_array)
    pre_anatomical = scaler.fit_transform(pre_anatomical.reshape(-1, pre_anatomical.shape[-1])).reshape(pre_anatomical.shape)
    pre_anatomical = np.stack([pre_anatomical], axis=3)
#     print(pre_anatomical.shape)
    np.save('input_folder/images/image_'+str(index)+'.npy', pre_anatomical)
    mri_volumes.append(pre_anatomical)
    
    # grouth preparing
    img_gt = nib.load(paths_gt[i])
    retreve_volume_gt = image.index_img(img_gt, 0)
    # Get data from nibabel image object 
    img_data_gt = retreve_volume_gt.get_data()
    #Convert to numpy ndarray (dtype: uint16)
    img_data_gt = np.asarray(img_data_gt)
    temp_mask = np.uint8(preprocess_ground_truth(img_data_gt))
    pre_gt = to_categorical(temp_mask, num_classes = 2)
#     print(pre_gt.shape)
    np.save('input_folder/masks/mask_'+str(index)+'.npy', pre_gt) # create a path for saving the files
    ground_truth_volumes.append(pre_gt)